In [1]:
# other libraries
import os
import glob
import shutil
from pathlib import Path
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import folium
#import warnings
#warnings.filterwarnings("ignore")      
#import sys

# geospatial packages
# from osgeo import gdal, ogr, osr
import geopandas as gpd
from shapely.geometry import Polygon
import rasterio as rio
from ultralytics import YOLO

In [13]:
path_to_model = "..\\models\\my_annotations\\my_annotations_yolov8m.pt_640\\weights\\best.pt"

train_config = "..\\data\\annotated_data\\train\\my_annotations\\train_config.yaml"
project = "..\\models\\my_annotations\\my_annotations_yolov8m.pt_640"
conf = 0.60

path_to_tiles = "..\\data\\tiles\\test_data\\10m_braatan_40m_20230605_sun"

In [11]:
path_to_tiles

'..\\data\tiles\test_data\x08m_braatan_40m_20230605_sun'

In [8]:
model = YOLO(path_to_model)
model

In [ ]:
model.predict(
    source=path_to_tiles,
    project=project,
    conf=conf,
    save=True,
    save_txt=True,
    save_conf=True,
    line_width=1
)

## Post-processing

In [24]:
gtiffs = glob.glob(f"{path_to_tiles}\\*.tif")
labels = glob.glob(f"{project}\\predict\\labels\\*.txt")
print(f"There a total of {len(gtiffs)} .tif files")
print(f"There a total of {len(labels)} .label files")

There a total of 416 .tif files
There a total of 379 .label files


In [28]:
# load tile index
path_to_tile_index = f"{path_to_tiles}//braatan_40m_20230605_sun_tile_index.shp"
tile_index = gpd.read_file(path_to_tile_index)
tile_index

,ID,geometry
0,braatan_40m_20230605_sun_01_01.tif,"POLYGON ((610764.790 6622705.168, 610774.794 6..."
1,braatan_40m_20230605_sun_01_02.tif,"POLYGON ((610773.791 6622705.168, 610783.795 6..."
2,braatan_40m_20230605_sun_01_03.tif,"POLYGON ((610782.792 6622705.168, 610792.796 6..."
3,braatan_40m_20230605_sun_01_04.tif,"POLYGON ((610791.793 6622705.168, 610801.797 6..."
4,braatan_40m_20230605_sun_01_05.tif,"POLYGON ((610800.794 6622705.168, 610810.798 6..."
...,...,...
1491,braatan_40m_20230605_sun_34_40.tif,"POLYGON ((611115.829 6622408.136, 611125.833 6..."
1492,braatan_40m_20230605_sun_34_41.tif,"POLYGON ((611124.830 6622408.136, 611134.834 6..."
1493,braatan_40m_20230605_sun_34_42.tif,"POLYGON ((611133.831 6622408.136, 611143.835 6..."
1494,braatan_40m_20230605_sun_34_43.tif,"POLYGON ((611142.832 6622408.136, 611152.836 6..."


In [30]:
from osgeo import gdal, osr

In [31]:
# Get raster metadata
# pixel resolution (in meters) and tile size in pixels
src_ds = gdal.Open(gtiffs[0]) # get raster datasource
_, xres, _, _, _, yres  = src_ds.GetGeoTransform() # get pixel size in meters
tile_size_m=round(src_ds.RasterXSize*xres)
tile_size_px= round(tile_size_m/abs(xres)) # calculate the tile size in pixels
## Get EPSG code
proj = osr.SpatialReference(wkt=src_ds.GetProjection())
EPSG_code= proj.GetAttrValue('AUTHORITY',1)

print("Resolution: "+str(round(xres,2))+" m")
print("EPSG: "+str(EPSG_code))

Resolution: 0.01 m
EPSG: 25832


c:\NMBU\Miniconda3\envs\deep_learning\Lib\site-packages\osgeo\gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [32]:
def yolo2xy(label_file, img_width, img_height):
    """
    Definition: 
Parameters: 
"""
    lfile = open(label_file)
    coords = []
    all_coords = []
    for line in lfile:
        l = line.split(" ")
        label=list(map(float, list(map(float, l[0]))))
        coords = list(map(float, list(map(float, l[1:6]))))
        x1 = float(img_width) * (2.0 * float(coords[0]) - float(coords[2])) / 2.0
        y1 = float(img_height) * (2.0 * float(coords[1]) - float(coords[3])) / 2.0
        x2 = float(img_width) * (2.0 * float(coords[0]) + float(coords[2])) / 2.0
        y2 = float(img_height) * (2.0 * float(coords[1]) + float(coords[3])) / 2.0
        tmp = [int(label[0]), int(x1), int(y1), int(x2), int(y2), float(coords[4])]
        all_coords.append(list(tmp))
    lfile.close()
    return all_coords

In [77]:
l = labels[0]

In [89]:
    r = gdal.Open(gtiff_file)
    _, xres, _, _, _, yres  = r.GetGeoTransform() # get pixel size in meters
    proj = osr.SpatialReference(wkt=src_ds.GetProjection())
    EPSG_code = proj.GetAttrValue('AUTHORITY',1)
    img_width = r.RasterXSize
    img_height = r.RasterYSize


In [91]:
all_bboxes = []
for l in labels:
    coords = yolo2xy(l, img_width, img_height)
    label_file_name=Path(l).stem
    gtiff_file = gtiffs[label_file_name in gtiffs]

    r = gdal.Open(gtiff_file)
    _, xres, _, _, _, yres  = r.GetGeoTransform() # get pixel size in meters
    proj = osr.SpatialReference(wkt=src_ds.GetProjection())
    EPSG_code = proj.GetAttrValue('AUTHORITY',1)
    img_width = r.RasterXSize
    img_height = r.RasterYSize

    tile = tile_index[tile_index['ID'] == label_file_name+".tif"]

    ## get tile bounding box geographical coordinates (UTM)
    tile_Xmin, tile_Ymin, tile_Xmax, tile_Ymax = tile.total_bounds

    ## take inner buffer equal to the buffer_size_m (remove overlap)
    tile_inner = tile
    tile_inner.loc[:, "geometry"] = tile_inner.geometry.buffer(-(buffer_size_m / 2))

    ## get inner tile bounding boxes
    inner_Xmin, inner_Ymin, inner_Xmax, inner_Ymax = tile_inner.total_bounds

    # Create box for each observation.
    bboxes = []
    for i in coords:
        X1 = (i[1] * xres) + tile_Xmin
        Y1 = (i[2] * yres) + tile_Ymin + tile_size_m
        X2 = (i[3] * xres) + tile_Xmin
        Y2 = (i[4] * yres) + tile_Ymin + tile_size_m

        # skip bounding box if its centroid is NOT within the inner tile (removing the overlap)
        X = (X1 + X2) / 2
        Y = (Y1 + Y2 )/2
        if (
            X < inner_Xmin or
            X > inner_Xmax or
            Y < inner_Ymin or
            Y > inner_Ymax):
            #print("continue break")
            continue
        # Create polygon shape from geographical coords
        lat_point_list = [Y1, Y1, Y2, Y2, Y1]
        lon_point_list = [X1, X2, X2, X1, X1]
        polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
        crs = {'init': 'epsg:'+ EPSG_code}
        data= {'class': [i[0]], 'prob': [i[5]]}
        bbox = gpd.GeoDataFrame(data, crs=crs, geometry=[polygon_geom])

        bboxes.append(bbox)
        bboxes_tile = gpd.GeoDataFrame(pd.concat(bboxes, ignore_index=True))
    all_bboxes.append(bboxes_tile)

c:\NMBU\Miniconda3\envs\deep_learning\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\NMBU\Miniconda3\envs\deep_learning\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\NMBU\Miniconda3\envs\deep_learning\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initializatio

In [90]:
bounding_boxes = gpd.GeoDataFrame(pd.concat(all_bboxes, ignore_index=True))
bounding_boxes